## GPUs: From Pipelines to Accelerators

A GPU is:
* Specialized hardare for graphics rendering
* Optimized for floating point transformations on 
    * 3-d models (representations of physical objects)
    * 2-d dense arrays (presentation) 
    
    
_A little history_. The GPU pipeline evolved from customized hardware for 3-d rendering onto 2-d planes for visualization.  The translation of the representation of a physical system into monitor/screen views.
  * Input: vertexes and primitves, lighting parameters, transformations
  * Output: 2-d image for display
  
The pipeline was fixed function transforming models into images.
The user prepared and loaded data then invoked the pipeline with DirectX or OpenGL.
The pipeline had phases:
* pipeline:
  * triangulate polygons
  * prepare vertex data streams
  